#**Importing Libraries and data**

In [2]:
# Library for installing module to check fastText vectorization
! pip install sister

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 674 kB 4.9 MB/s 
     |████████████████████████████████| 753.2 MB 12 kB/s 
     |████████████████████████████████| 24.2 MB 1.4 MB/s 
     |████████████████████████████████| 17.1 MB 17.5 MB/s 
     |████████████████████████████████| 21.5 MB 142.3 MB/s 
     |████████████████████████████████| 68 kB 6.4 MB/s 
     |████████████████████████████████| 14.6 MB 20.0 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
     |████████████████████████████████| 880 kB 59.5 MB/s 
     |████████████████████████████████| 1.3 MB 70.7 MB/s 
     |████████████████████████████████| 7.5 MB 47.6 MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=3124694 sha256=69f4baccb0410b5bdd599cedd2d908ed347b9a91c9ff3abcdd2eff35d6585278
  Stored in directory: /root/.cache/pip/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f

In [3]:
# Importing all the necessary Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import sister

In [4]:
# Importing the files
f_data = pd.read_csv("flipkart_com-ecommerce_sample.csv", engine = "python", encoding = "utf-8")
a_data = pd.read_csv("amz_com-ecommerce_sample.csv", engine = 'python',encoding = "latin")

In [5]:
# Seeing the amazon files
a_data.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32143,29121,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,208,258,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [6]:
# Seeing the flipkart files
f_data

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,7179d2f6c4ad50a17d014ca1d2815156,2015-12-01 10:15:43 +0000,http://www.flipkart.com/walldesign-small-vinyl...,WallDesign Small Vinyl Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7KFJAKSTDY9G,1500.0,730.0,"[""http://img6a.flixcart.com/image/wall-decorat...",False,Buy WallDesign Small Vinyl Sticker for Rs.730 ...,No rating available,No rating available,WallDesign,"{""product_specification""=>[{""key""=>""Number of ..."
19996,71ac419198359d37b8fe5e3fffdfee09,2015-12-01 10:15:43 +0000,http://www.flipkart.com/wallmantra-large-vinyl...,Wallmantra Large Vinyl Stickers Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE9F5URNQGJCGH,1429.0,1143.0,"[""http://img6a.flixcart.com/image/sticker/z/g/...",False,Buy Wallmantra Large Vinyl Stickers Sticker fo...,No rating available,No rating available,Wallmantra,"{""product_specification""=>[{""key""=>""Number of ..."
19997,93e9d343837400ce0d7980874ece471c,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7VAYDKQZEBSD,1299.0,999.0,"[""http://img5a.flixcart.com/image/sticker/b/s/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ..."
19998,669e79b8fa5d9ae020841c0c97d5e935,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,Elite Collection Medium Acrylic Sticker,"[""Baby Care >>

Since we have to compare the prices of 2 datasets by entering name of one therefore I have only selected one feature product name to find the similar product from other datasets . We can improve further by taking brand name also as feature to improve further.

# **Pre-processing**

In [7]:
# Taking product name , retail price and discounted price as features and making another datasets.
f_data1 = f_data[['product_name','retail_price','discounted_price']].copy()
a_data1 = a_data[['product_name','retail_price','discounted_price']].copy()

In [8]:
# cleaning the data to remove the unnecessary signs,spaces etc
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower() 
# Applying the function to the data 
f_data1['product_name1'] = f_data1['product_name'].map(preprocess)
a_data1['product_name1'] = a_data1['product_name'].map(preprocess)

In [9]:
# Making the joint corpus of both flipkart and amazon 
corpus_flipkart = list(f_data1['product_name1'].values)
corpus_amz = list(a_data1['product_name1'].values)
corpus = corpus_flipkart + corpus_amz

In [10]:
# Vectorizing the data of both amazon and flipkart data i.e corpus
vectorizer = TfidfVectorizer(stop_words='english')
encoding = vectorizer.fit(corpus)
# transforming the data of both data of amazon and flipkart
X = encoding.transform(corpus_flipkart)
Y = encoding.transform(corpus_amz)

In [11]:
X.shape

(20000, 12218)

# Enter the text to be searched here

In [13]:
# Enter the data to search
user_data = "Walldesign sticker" #@param {type:"string"}
# Now pre-processing the data entered
user_data = preprocess(user_data)
pre_user_data = encoding.transform(list([user_data]))

In [18]:
pre_user_data

<1x12218 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [27]:
# deciding the treshold
threshold = 0.7

In [23]:
cosine_similarity(pre_user_data,Y[19799])

array([[0.82381336]])

# **Function to find similar products**

### Running this cell will bring all the product similar to entered data from **Amazon**

In [28]:
# Program to take similar products from the input data from amazon datasets
s = []
for y in range(0,Y.shape[0]):
  if(((cosine_similarity(pre_user_data,Y[y]))[0]>threshold)):
    s.append(y)
# Now print all the similar objects
print(a_data1[['product_name','retail_price','discounted_price']].iloc[s,])

                         product_name  retail_price  discounted_price
16424          WallDesign PVC Sticker          5883              6515
19799      WALLDESIGN S VINYL STICKER          1495               934
19801      WALLDESIGN S VINYL STICKER          1492               922
19807      WALLDESIGN S VINYL STICKER          1490               922
19808      WALLDESIGN S VINYL STICKER          1205               693
...                               ...           ...               ...
19981  WALLDESIGN SMALL VINYL STICKER          1078               661
19986  WALLDESIGN SMALL VINYL STICKER          1205               701
19988  WALLDESIGN SMALL VINYL STICKER          1291               797
19991  WALLDESIGN SMALL VINYL STICKER          1212               684
19995  WALLDESIGN SMALL VINYL STICKER          1498               876

[66 rows x 3 columns]


## Running this cell will bring all the similar entered data from **Flipkart**

In [29]:
# Program to take similar products from the input data from flipkart datasets
v = []
for p in range(0,X.shape[0]):
  if(((cosine_similarity(pre_user_data,X[p]))[0]>threshold)):
    v.append(p)
# Now print all the similar objects
print(f_data1[['product_name','retail_price','discounted_price']].iloc[v,])

                         product_name  retail_price  discounted_price
16424          WallDesign PVC Sticker        5899.0            5279.0
19799  WallDesign Small Vinyl Sticker        1500.0             730.0
19801  WallDesign Small Vinyl Sticker        1500.0             730.0
19807  WallDesign Small Vinyl Sticker        1500.0             730.0
19808  WallDesign Small Vinyl Sticker        1220.0             600.0
...                               ...           ...               ...
19981  WallDesign Small Vinyl Sticker        1080.0             530.0
19986  WallDesign Small Vinyl Sticker        1220.0             600.0
19988  WallDesign Small Vinyl Sticker        1300.0             640.0
19991  WallDesign Small Vinyl Sticker        1220.0             600.0
19995  WallDesign Small Vinyl Sticker        1500.0             730.0

[66 rows x 3 columns]


Above is the final model we are using for the searching the database

Below is another model I give it a try but couldnot bring more accuracy in this time period

# **Using FastText encoding(not the best method found)**

It is the latest encoding teqnique that uses small world syllables to encode the word and is better than Tf-Id vectorizer. As it is most use for classification and thats we dont use here  in this problems thats why we dont use it here.

In [30]:
# First we will make embedding function.This MeanEmbedding uses FastText method to embed sentences in 300 features.This is pretrained model which is downloaded
# from net
embedder = sister.MeanEmbedding(lang="en")

Loading model...


In [ ]:
# This is encoding the user input into fastText method
em_user_data = embedder(user_data)

In [ ]:
# The user_data shape is this we have to change into (1,300)
em_user_data.shape

(300,)

Now we are finding the entered data into amazon and flipkart datasets using Fasttexts.

In [ ]:
# This is program to find the data from amazon datasets
k = []
for y in range(0,Y.shape[0]):
  if(((cosine_similarity(em_user_data.reshape(1,-1),embedder(corpus_amz[y]).reshape(1,-1)))[0]>0.5)):
    k.append(y)
# Now print all the similar objects
print(a_data1[['product_name','retail_price','discounted_price']].iloc[k,])

                                     product_name  retail_price  \
1             FabHomeDecor Fabric Double Sofa Bed         32143   
7             FabHomeDecor Fabric Double Sofa Bed         32137   
16            FabHomeDecor Fabric Double Sofa Bed         32150   
19            FabHomeDecor Fabric Double Sofa Bed         32144   
20     SICONS CONDITIONING CONDITONER DOG SHAMPOO            97   
...                                           ...           ...   
19949                  WALLDESIGN S VINYL STICKER          1290   
19954    WALLMANTRA MEDIUM VINYL STICKERS STICKER           711   
19961    WALLMANTRA MEDIUM VINYL STICKERS STICKER           772   
19962                       STYLISTRY WOMEN HEELS          1784   
19971    WALLMANTRA MEDIUM VINYL STICKERS STICKER           772   

       discounted_price  
1                 29121  
7                 28664  
16                28650  
19                26423  
20                  119  
...                 ...  
19949        

In [ ]:
# This is programs to find data from flipkart datasets.
f = []
for y in range(0,X.shape[0]):
  if(((cosine_similarity(em_user_data.reshape(1,-1),embedder(corpus_flipkart[y]).reshape(1,-1)))[0]>0.8)):
    f.append(y)
# Now print all the similar objects
print(a_data1[['product_name','retail_price','discounted_price']].iloc[f,])

Empty DataFrame
Columns: [product_name, retail_price, discounted_price]
Index: []


As we can see that the Fasttext model which we are using is trained on wikipedia therefore it is not able to make good prediction on amazon and flipkart playlist.If we train it on there datasets then we can improve fastText model predicrtion more higher than Tf-Id Vectorizer.